# EMODnet Human Activites tutorial/Quickstart

## 1.1 Setup ODP Client

In [ ]:
!pip install -U odp-sdk --quiet && pip list | grep odp
!pip install -U pydeck --quiet && pip list | grep pydeck

In [ ]:
from odp.client import OdpClient # The SDK
import pandas as pd
import pydeck as pdk
from helper_functions import convert_geometry_field, wkt_to_edges

### 1.1.2 Geometric Plot Helper

In [ ]:
# loading a simple function to make a nice map display of points
def simple_pydeck_plot(geojson_lists, size=1, lat=0, lon=0, zoom=2, fillopacity=0.9):
    colors = [(3, 255, 209), (157, 89, 244), (254, 119, 76), (255, 216, 11)]  # Cyan, Yellow, Purple, Orange
    layers = []
    for i, geojson_list in enumerate(geojson_lists):
        layer = pdk.Layer("GeoJsonLayer", data=convert_geometry_field(geojson_list), opacity=fillopacity, lineWidthMinPixels=size, pointRadiusMinPixels=size, pickable=True, get_line_color=colors[i % len(colors)], get_fill_color=colors[i % len(colors)],)
        layers.append(layer)
    r = pdk.Deck(layers=layers, initial_view_state=(pdk.ViewState(latitude=lat, longitude=lon, zoom=zoom)), map_provider="mapbox", map_style='mapbox://styles/oceandatafoundation/clwg6xklg00an01pcgmeufjxq', api_keys={'mapbox': 'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'})
    return r

In [ ]:
client = OdpClient()


### 2.1 Fetch Metadata from Catalog
 - Option 1) via API calls to the catalog
 - Option 2) getting UUID or qualified name from the landing page:
https://app.hubocean.earth/catalog/dataset/ec6bd5ff-1713-4703-8261-caa2de45953c
![image.png](attachment:2549655c-c3b0-4240-952c-97a4ef791e6e.png)

In [ ]:
## Request the dataset from the catalog using the UUID:
OilWells = client.catalog.get("ec6bd5ff-1713-4703-8261-caa2de45953c")
OilWells.metadata.display_name


In [ ]:
Ports = client.catalog.get("03b0590a-7ddb-4828-8b11-06fb5c78cafe")
Ports.metadata.display_name


In [ ]:
TeleCables = client.catalog.get("3c55c6eb-45a2-4a72-8b65-4d9a5896a058")
TeleCables.metadata.display_name


### 2.2 Fetch data from the tabular v2

In [ ]:
OilWells_dataset = client.table_v2(OilWells)
pages = list(OilWells_dataset.select().pages())
OilWells_data = [row for page in pages for row in page]

The ports dataset is large, so we will only fetch 10 000 rows.

In [ ]:
Ports_dataset = client.table_v2(Ports)

Ports_data = []
cursor = Ports_dataset.select()
for i, row in enumerate(cursor.rows()):
    if i >= 10_000:  
        break
    Ports_data.append(row)

In [ ]:
TeleCables_dataset = client.table_v2(TeleCables)
pages = list(TeleCables_dataset.select().pages())
TeleCables_data = [row for page in pages for row in page]

Limit the Port data, as pydeck is unable to handle large amounts of data

In [ ]:
len(OilWells_data), len(Ports_data), len(TeleCables_data)

In [ ]:
Ports_data[0]

## 3.1 Visualize data

### 3.1.1 Plot datapoints from query on a map

In [ ]:
simple_pydeck_plot([Ports_data, OilWells_data, TeleCables_data], lat=55, lon=7, zoom=3)


## 4.1 Test a geospatial query of some points

### 4.1.1 Create geospatial query

In [ ]:
bbox_lat_min = 52
bbox_lat_max = 57
bbox_lon_min = 4
bbox_lon_max = 12

In [ ]:
from shapely.geometry import box, LineString, mapping

geospatial_query = box(
    bbox_lon_min, bbox_lat_min,
    bbox_lon_max, bbox_lat_max
).wkt
bbox_list = wkt_to_edges(geospatial_query)

In [ ]:
geospatial_query

## 4.2 Fetch filtered data

We query the data again; this time, we want all ports within the bounding box that are dated after 2015, or any ports located in Norway.

In [ ]:
filtered_pages = list(Ports_dataset.select(f"geometry within '{geospatial_query}' AND year > 2015 OR country == 'NO'").pages())
Ports_subset = [row for page in filtered_pages for row in page]

### 4.2.1 Plot filtered data with bounding box

In [ ]:
# comparing the full dataset to the geospatial subset
simple_pydeck_plot([Ports_subset, bbox_list], size=2, lat=55, lon=7, zoom=3)